In [1]:
import pandas as pd
import io
import re
import csv
import json
import numpy as np
from tqdm import tqdm
import math
import ast
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import seaborn as sns
import keras
from keras import backend as K
from progressbar import ProgressBar
pbar = ProgressBar()
import matplotlib.pyplot as plt
from keras import Sequential, Model


from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing

tqdm.pandas()

Using TensorFlow backend.


In [2]:
max_blocks = 37 #(36 + 1 FC layer at the end)
#max_blocks = 36

nb_param =13
#nb_param =12

BATCH_SIZE = 2048
SHUFFLE_BUFFER_SIZE = 100

In [3]:

def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.asarray(ast.literal_eval(array_string))

# Load Proccessed Neural Networks (without HyperParameters)

In [4]:

path_raw_val = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val1.csv'
path_raw_train = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train1.csv'


path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/val_7.csv'
path_processed_train_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/train_7.csv'

In [5]:
#%pycache

val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})
train = pd.read_csv(path_processed_train_nn,converters={'NN_dataframe': from_np_array})

# Complete to zeros (max shape)# Complete to zeros (max shape)

In [6]:
def add_zero_blocks(arr):
    zero_blocks = np.zeros((max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)

val['NN_dataframe']= val['NN_dataframe'].apply(lambda x : add_zero_blocks(x))
train['NN_dataframe']= train['NN_dataframe'].apply(lambda x : add_zero_blocks(x))




# Get y_train, y_val


In [7]:
# To retrieve y_train / y_test

train_hw = pd.read_csv(path_raw_train)
val_hw = pd.read_csv(path_raw_val)

y_train = np.array(train_hw["total_power"].tolist())
y_train=y_train.reshape(y_train.shape[0],-1)

y_val = np.array(val_hw["total_power"].tolist())
y_val=y_val.reshape(y_val.shape[0],-1)

--------
# HW PARAM ALONE

In [8]:
X_train_hw_param  = train_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_train_hw_param['tot_mac'] = X_train_hw_param['mac_num']*X_train_hw_param['mac_array_num']

X_val_hw_param  = val_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_val_hw_param['tot_mac'] = X_val_hw_param['mac_num']*X_val_hw_param['mac_array_num']

X_train_hw_param.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac
0,118.0,2.0,256.0,179968.0,2048.0,236.0
1,87.0,2.0,512.0,99968.0,2048.0,174.0
2,124.0,2.0,512.0,99968.0,3072.0,248.0
3,80.0,2.0,1024.0,99968.0,1024.0,160.0
4,86.0,2.0,512.0,60000.0,1536.0,172.0


## HW Normalize

In [9]:
# normalize

#%pycache
# Process
scaler = preprocessing.StandardScaler().fit(X_train_hw_param)
X_train_hw_param_norm= scaler.transform(X_train_hw_param)
X_val_hw_param_norm = scaler.transform(X_val_hw_param )


### HW Dataset

In [10]:
hw_train_dataset = tf.data.Dataset.from_tensor_slices((np.array(X_train_hw_param), y_train))
hw_test_dataset = tf.data.Dataset.from_tensor_slices((np.array(X_val_hw_param), y_val))

hw_train_dataset_norm = tf.data.Dataset.from_tensor_slices((np.array(X_train_hw_param_norm), y_train))
hw_test_dataset_norm = tf.data.Dataset.from_tensor_slices((np.array(X_val_hw_param_norm), y_val))


BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = 100

hw_train_dataset = hw_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
hw_test_dataset = hw_test_dataset.batch(BATCH_SIZE)

------------

# NN TREATEMENT ALONE

## Data preprocessing

In [11]:
X_train_nn =np.array(train['NN_dataframe'].tolist())
X_val_nn = np.array(val['NN_dataframe'].tolist())

In [12]:

#Reshape
train_reshape= X_train_nn.reshape(X_train_nn.shape[0], -1)
val_reshape= X_val_nn.reshape(X_val_nn.shape[0], -1)

# Process
scaler = preprocessing.StandardScaler().fit(train_reshape)
train_scaled= scaler.transform(train_reshape)
val_scaled = scaler.transform(val_reshape)

#Re-reshape
X_train_nn_norm= train_scaled.reshape(X_train_nn.shape)
X_val_nn_norm= val_scaled.reshape(X_val_nn.shape)

## Dataset on NN features

### Normalized

In [13]:
#%pycache

#define dataset
nn_train_dataset_norm = tf.data.Dataset.from_tensor_slices((X_train_nn_norm, y_train))
nn_test_dataset_norm = tf.data.Dataset.from_tensor_slices((X_val_nn_norm, y_val))

# shuffle batch
nn_train_dataset_norm = nn_train_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
nn_test_dataset_norm = nn_test_dataset_norm.batch(BATCH_SIZE)

### Not normalized

In [14]:
#define dataset
nn_train_dataset = tf.data.Dataset.from_tensor_slices((X_train_nn, y_train))
nn_test_dataset = tf.data.Dataset.from_tensor_slices((X_val_nn, y_val))

# shuffle batch
nn_train_dataset = nn_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
nn_test_dataset = nn_test_dataset.batch(BATCH_SIZE)


### Test dim

# Get loss/model

In [34]:
def loss(target_power, predicted_power):
  return tf.math.abs((target_power - predicted_power)/target_power)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=15*67,
    decay_rate=0.9)
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

## Models

In [35]:


model_nn= tf.keras.Sequential([
    
      layers.Conv1D(64, (3), activation='relu', padding="same",input_shape=(max_blocks, 7)),
      layers.Conv1D(32, (3), activation='relu', padding="same"),
      layers.Conv1D(32, (3), activation='relu', padding="same"),
      layers.Conv1D(1, (3), activation='relu', padding="same"),
      layers.Lambda( lambda x: K.sum(x, axis=1)),
     #layers.Flatten()
])


model_hw = tf.keras.Sequential([
     layers.Dense(32, activation='relu', input_shape=(6,)),
     layers.Dense(32, activation='relu'),
     layers.Dense(1, activation='linear')
 ])


In [39]:
concat = tf.keras.layers.Concatenate()([model_nn.output, model_hw.output])
#output = tf.keras.layers.Dense(units=10, activation='relu')(concat)
output = tf.keras.layers.Dense(units=1,activation='relu')(output)
full_model = tf.keras.Model(inputs=[model_nn.input, model_hw.input], outputs=[output])

In [40]:
full_model = tf.keras.Model(inputs=[model_nn.input, model_hw.input], outputs=[output])
full_model.compile(optimizer=optimizer, loss=loss)


In [42]:
#ull_model.fit([train_dataset_norm,hw_train_dataset], epochs=2)
history = full_model.fit([X_train_nn_norm,X_train_hw_param_norm], y=y_train, epochs=10, batch_size=2048, validation_data=([X_val_nn_norm, X_val_hw_param_norm], y_val))
#history = full_model.fit([X_train_nn_norm,X_train_hw_param], y=y_train, epochs=10, batch_size=2048, validation_data=([X_val_nn_norm, X_val_hw_param], y_val))



Train on 137090 samples, validate on 13804 samples
Epoch 1/10
 90112/137090 [==================>...........] - ETA: 7s - loss: 1.0000

KeyboardInterrupt: 

In [21]:
predictor =  full_model

nb_val = len(X_val_norm)
nb_pred = 40
start = 12980
a = []
preds = []
trues = []
index_low=[]
index_middle = []
index_high = []
for k in (range(nb_pred)):
    i=randrange(len(X_val_norm))
    t=np.array(X_val_norm)[i]
    t=t.reshape((1,max_blocks,nb_param))
    ex=tf.convert_to_tensor(t)
    pred = predictor.predict(ex)
    true = y_val[i]
    if true < 100:
        index_low.append(k)
    elif 100<=true <1000:
       index_middle.append(k)        
    else:
        index_high.append(k)
    print()
    print(i)
    preds.append(pred)
    trues.append(true)
    print(f'pred : {pred}')
    print(f'true : {true}')
    a.append(loss(true,pred))
    print(f'loss : {loss(true,pred)}')
print()
print(f'loss : {np.mean(a)}')


NameError: name 'full_model' is not defined

In [22]:
plt.figure()
plt.plot(np.array(preds).reshape(nb_pred,-1))
plt.plot(np.array(trues).reshape(nb_pred,-1))
plt.title(f'All predictions : mean loss = {round(np.mean(a),2)}')
plt.ylabel('predictions')
plt.xlabel('range')
plt.legend(['preds', 'trues'], loc='upper left')
plt.show()

plt.figure()
plt.plot(np.array(preds)[index_low].reshape(len(index_low),-1))
plt.plot(np.array(trues)[index_low].reshape(len(index_low),-1))
plt.title('Low Value Predictions')
plt.ylabel('predictions')
plt.xlabel('range')
plt.legend(['preds', 'trues'], loc='upper left')
plt.show()

plt.figure()
plt.plot(np.array(preds)[index_middle].reshape(len(index_middle),-1))
plt.plot(np.array(trues)[index_middle].reshape(len(index_middle),-1))
plt.title('Middel Value Predictions')
plt.ylabel('predictions')
plt.xlabel('range')
plt.legend(['preds', 'trues'], loc='upper left')
plt.show()

plt.figure()
plt.plot(np.array(preds)[index_high].reshape(len(index_high),-1))
plt.plot(np.array(trues)[index_high].reshape(len(index_high),-1))
plt.title('High Value Predictions')
plt.ylabel('predictions')
plt.xlabel('range')
plt.legend(['preds', 'trues'], loc='upper left')
plt.show()


NameError: name 'preds' is not defined